# Comparación Visual de las clasificaciones Supervisadas Aysén
Script trabajado por Francisco Albornoz (2021-05-03 10:47:09  ) / Última modificación / Comparación visual de las clasificaciones supervisadas región de Aysen

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

1) Importamos las librerías a utilizar

In [1]:
import os
import ee
import geemap

## Creamos un mapa interactivo 

Para crear un mapa interactivo asignamos a "Map" la función geemap.Map() que es la que contendrá el mapa interactivo, por lo que podemos crear varios mapas según nuestra necesidad. Para centrarlo automaticamente en la región de interés agregamos un punto con sus coordenadas para centrar nuestra visualización.

In [2]:
Map = geemap.Map()
centroid = ee.Geometry.Point([-72, -45]) #Coordenada de área de interés
Map.centerObject(centroid, 8) #Número 8 representa el zoom del mapa
Map #activamos el mapa para visualizar

Map(center=[-45, -72], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Inicializamos los mapas linkeados

A continuación se mostrará un ejemplo de los mapas linkeados de una imagen de Landsat 8 para la región de Aysén.
En diferentes orden de bandas.

In [3]:
geemap.ee_initialize()

In [4]:
landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(centroid) \
    .filterDate('2020-01-01', '2021-03-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

In [5]:
vis_params = [
    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1}, 
    {'bands': ['B6', 'B5', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1},
    {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 3000, 'gamma': 1},
    {'bands': ['B5', 'B6', 'B4'], 'min': 0, 'max': 3000, 'gamma': 1}
]
vis_params_mosaico = [
    {'bands': ['b3', 'b2', 'b1'], 'min': 0, 'max': 0.2, 'gamma': 1}, 
    {'bands': ['b7'],'min': 0, 'max': 1, 'palette' : ['0000FF', 'FF0000']},
    {'bands': ['b4', 'b3', 'b2'], 'min': 0, 'max': 0.5, 'gamma': 1},
    {'bands': ['b6', 'b7', 'b5'], 'min': 0, 'max': 1, 'gamma': 1}
]

In [6]:
labels = [
    'Natural Color (B4/B3/B2)',
    'Agricultura (B6/B5/B2)',
    'Color Infrarojo (B5/B4/B3)',
    'Masas de agua (B5/B6/B4)'
]
labelsmosaico = [
    'Natural Color (B4/B3/B2)',
    'NDVI',
    'Color Infrarojo (B5/B4/B3)',
    'Vegetación sana'
]

In [7]:
geemap.linked_maps(rows=2, cols=2, height="400px", center=[-45, -72], zoom=8, 
                   ee_objects=[landsat], vis_params=vis_params, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

## Inicializamos las clasificaciones 

Primero importamos la imagen a clasificar y nuestras áreas de entrenamiento. Para luego extraer los valores a dicha imagen, de esta manera tendremos un vector por pixel que contendrá los valores de las bandas de la imagen landsat a clasificar, más los valores de nuestros set de entrenamientos, para que la clasificación resulte dichos valores deben ser números ya que con palabras o strings el set de comandos no lo reconocerá.

In [8]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")

In [9]:
clas18c = ee.Image("users/franciscoalbornoz/clasificacion_18c")


In [10]:
clas18c.bandNames().getInfo()

['b1']

In [11]:
mosaico.bandNames().getInfo()

['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

Importamos nuestros shapes de entrenamiento

In [12]:
work_dir = os.path.expanduser('../Data/shapes/')

val_shp= os.path.join(work_dir, 'validacion.shp')
merged_original =os.path.join(work_dir,'roi_merged.shp')

In [13]:
mergedoriginal =geemap.shp_to_ee(merged_original)
Map.addLayer(mergedoriginal, {}, 'Áreas de entrenamiento Original')

Extraemos los valores de nuetra imagen landsat según nuestras muestras de entrenamiento

In [14]:
dataset1 = mosaico.sampleRegions(**{
    'collection': mergedoriginal,
    'properties': ['ID'],
    'scale': 30,
})

In [15]:
val = geemap.shp_to_ee(val_shp)
Map.addLayer(val, {}, 'Áreas de Validación')

In [16]:
datasetval = mosaico.sampleRegions(**{
    'collection': val,
    'properties': ['ID'],
    'scale': 10,
})

In [17]:
Map

Map(center=[-45, -72], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [18]:
# Guardamos las bandas
band = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']
# Esta propiedad guarda las clases de landcover
label1 ='ID'

### Random Forest

In [19]:
classifier100 = ee.Classifier.smileRandomForest(100).train(dataset1, label1, bands) # 100 árboles de desición

In [20]:
resultrandomforest = mosaico.select(bands).classify(classifier100)

### Support Vector Machine

In [21]:
classifiVM = ee.Classifier.libsvm('Voting','C_SVC','RBF', gamma = 1 , cost = 10 )

In [22]:
entrenamiento = classifiVM.train(dataset1, label1, bands)
classifiedVM = mosaico.classify(entrenamiento)

### Clasificación y árboles de Regresión

In [23]:
trained1 = ee.Classifier.smileCart().train(dataset1, label1, bands)

In [24]:
classifiedCART = mosaico.select(bands).classify(trained1)

### Mínima Distancia

In [25]:
classifierMD = ee.Classifier.minimumDistance("euclidean").train(dataset1, label1, bands)

In [26]:
resultMD = mosaico.select(bands).classify(classifierMD)

### Dando formato a los mapas clasificados

In [27]:
class_values1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18]

In [28]:
class_palette1 = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]
class_palette = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'c0c233'
]

## Visualizando las clasificaciones

In [29]:
landcover1 = resultrandomforest.set('classification_class_values', class_values1)
landcover1 = landcover1.set('classification_class_palette', class_palette1)

In [30]:
landcover2 = classifiedVM.set('classification_class_values', class_values1)
landcover2 = landcover2.set('classification_class_palette', class_palette1)

In [31]:
landcover3 = classifiedCART.set('classification_class_values', class_values1)
landcover3 = landcover3.set('classification_class_palette', class_palette1)

In [32]:
landcover4 = resultMD.set('classification_class_values', class_values1)
landcover4 = landcover4.set('classification_class_palette', class_palette1)

In [33]:
resultados = [landcover1, landcover2, landcover3, landcover4]

In [34]:
vis_params2 = [
    {}, 
    {},
    {},
    {},
]

In [35]:
labels2 = [
    'Random Forest',
    'Support Vector Machine',
    'Regressions Trees',
    'Mínima distancia', 
]

In [36]:
geemap.linked_maps(rows=2, cols=2, height="400px", center=[-45, -72], zoom=8, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels2, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [37]:

clas18cs_vis = {
    'min': 0,
    'max': 20,
    'bands': ['b1'],
    'palette' : ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]}
Map.addLayer(clas18c, clas18cs_vis  , name = 'Land cover clasificado CIEP')
Map

Map(center=[-45, -72], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [38]:
geemap.linked_maps(rows=2, cols=2, height="400px", center=[-45, -72], zoom=8, 
                   ee_objects=[mosaico], vis_params=vis_params_mosaico, labels=labelsmosaico, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [39]:
left_layer = geemap.ee_tile_layer(landcover1, {}, 'Random Forest')
right_layer = geemap.ee_tile_layer(mosaico, landsat_vis, 'Mosaico')

In [40]:
left_layer2 = geemap.ee_tile_layer(landcover2, {}, 'Support Vector Machine')

In [41]:
left_layer3 = geemap.ee_tile_layer(landcover3, {}, 'Regressions Trees')

In [42]:
left_layer4 = geemap.ee_tile_layer(landcover4, {}, 'Mínima distancia')

In [43]:
legend_dict = {
    '01 Achaparrado': '22c129',
    '02 Agrícola': 'f0ff00',
    '03 Agua': '0c40f1',
    '04 Arenal o Roca': '685205',
    '05 Bosque Degradado': '0d4404',
    '06 Cuncunas': 'e82d13',
    '07 Estepa': 'f99504',
    '08 Humedal': '17d4c6',
    '09 Mallin': '1c6330',
    '10 Matorral abierto': '24ff00',
    '11 Matorral denso': '2de50f',
    '12 Plantaciones': 'ccba7d',
    '13 Praderas': 'dec61a',
    '14 Primario': '28d909',
    '15 Ríos': '03c2ff',
    '16 Secundario': '4e8c44',
    '17 Sombra topo': 'fdfdfd',
    '18 Suelo desnudo': 'c0c233'
}

In [44]:
Map2 = geemap.Map()
Map2.centerObject(centroid, 8)
Map2.split_map(left_layer, right_layer)
Map2.add_legend(legend_title="Random Forest", legend_dict=legend_dict)
Map2

Map(center=[-45, -72], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [45]:
Map3 = geemap.Map()
Map3.centerObject(centroid, 8)
Map3.split_map(left_layer2, right_layer)
Map3.add_legend(legend_title="SVM", legend_dict=legend_dict)
Map3

Map(center=[-45, -72], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [46]:
Map4 = geemap.Map()
Map4.centerObject(centroid, 8)
Map4.split_map(left_layer3, right_layer)
Map4.add_legend(legend_title="SCART", legend_dict=legend_dict)
Map4

Map(center=[-45, -72], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [47]:
Map5 = geemap.Map()
Map5.centerObject(centroid, 8)
Map5.split_map(left_layer4, right_layer)

Map5.add_legend(legend_title="Mínima Distancia", legend_dict=legend_dict)
Map5


Map(center=[-45, -72], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…